In [1]:
#%%sh
#apt-get -y install unzip # Only needed in SageMaker Studio
# wget -N https://sagemaker-sample-data-us-west-2.s3-uswest-2.amazonaws.com/autopilot/direct_marketing/bankadditional.zip
# unzip -o bank-additional.zip

# above link not working thats why

# uploaded data manually
import os
os.listdir(".")

['test',
 'bank-additional-full.csv',
 '.cache',
 'Final.ipynb',
 '.ipynb_checkpoints',
 'train',
 '.jupyter',
 '.ipython',
 '.local',
 '.yarnrc',
 'preprocessing.py']

In [2]:
import pandas as pd
df = pd.read_csv('bank-additional-full.csv',sep = ';')
df.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


Now, let's upload the dataset to Amazon S3. We'll use a default bucket automatically
created by SageMaker in the region we're running in. We'll just add a prefix to keep
things nice and tidy

In [3]:
# jb file yaha agai hai to upload kiu kare wapis ?

import sagemaker
prefix = 'sagemaker/DEMO-smprocessing/input'
input_data = sagemaker.Session().upload_data(path='./bank-additional-full.csv',
                                             key_prefix=prefix)

In [4]:
# just writing
# Will execute it later
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--train-test-split-ratio',type=float, default=0.3)
args, _ = parser.parse_known_args()
print('Received arguments {}'.format(args))

split_ratio = args.train_test_split_ratio

Received arguments Namespace(train_test_split_ratio=0.3)


We load the input dataset using pandas. At startup, SageMaker Processing
automatically copied it from S3 to a user-defined location inside the container; here,
it is /opt/ml/processing/input:

In [1]:
import os
import pandas as pd
input_data_path = os.path.join('/opt/ml/processing/input', 'D:/Bank Additional/bank-additional/bank-additional-full.csv')
df = pd.read_csv(input_data_path,sep =";")

In [2]:
# Then, we remove any line with missing values, as well as duplicate lines:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [3]:
#Then, we count negative and positive samples, and display the class ratio.
#This will tell us how unbalanced the dataset is :

one_class = df[df['y']=='yes']
one_class_count = one_class.shape[0]
zero_class = df[df['y']=='no']
zero_class_count = zero_class.shape[0]
zero_to_one_ratio = zero_class_count/one_class_count
print("Ratio: %.2f" % zero_to_one_ratio)

Ratio: 7.88


In [4]:
import numpy as np
df['no_previous_contact'] = np.where(df['pdays'] == 999, 1, 0)

In [5]:
df['not_working'] = np.where(np.in1d(df['job'], ['student', 'retired', 'unemployed']), 1, 0)

In [10]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(df.drop('y', axis=1),df['y'],
test_size=split_ratio, random_state=0)

NameError: name 'split_ratio' is not defined

In [9]:
# not working
#from sklearn.compose import make_column_transformer
#from sklearn.preprocessing import StandardScaler,OneHotEncoder

#preprocess = make_column_transformer((['age', 'duration', 'campaign', 'pdays', 'previous'],
#                                      StandardScaler()),(['job', 'marital', 'education', 'default', 'housing',
#                                      'loan','contact', 'month', 'day_of_week','poutcome'],
#                                       OneHotEncoder(sparse=False)))
                                       
#train_features = preprocess.fit_transform(X_train)
#test_features = preprocess.transform(X_test)

In [10]:
# working
# Code link
# https://stackoverflow.com/questions/43798377/one-hot-encode-categorical-variables-and-scale-continuous-ones-simultaneouely

 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder

columns_to_scale = ['age', 'duration', 'campaign', 'pdays', 'previous']
columns_to_encode=['job', 'marital', 'education', 'default', 'housing','loan','contact', 'month', 'day_of_week','poutcome']


# Instantiate encoder/scaler
scaler = StandardScaler()
ohe    = OneHotEncoder(sparse=False)

p = Pipeline(
    [("coltransformer", ColumnTransformer(
        transformers=[
            ("assessments", Pipeline([("scale", scaler)]), columns_to_scale),
            ("ranks", Pipeline([("encode", ohe)]), columns_to_encode),
        ]),
    )]
)

#print(pd.DataFrame(p.fit_transform(df)).head())
#new_df = pd.DataFrame(p.fit_transform(df))
#new_df.head(2)
#train_features = pd.DataFrame(p.fit_transform(X_train))
#test_features = pd.DataFrame(p.fit_transform(X_test))

In [11]:
train_features = p.fit_transform(X_train)
test_features = p.transform(X_test)

In [12]:
################ Above code are working fine #################################
#https://github.com/drbilal216/amazon-sagemaker-examples/blob/master/aws_sagemaker_studio/sagemaker-python-sdk/scikit_learn_iris/Scikit-learn%20Estimator%20Example%20With%20Batch%20Transform.ipynb   

In [13]:
# Get a SageMaker-compatible role used by this Notebook Instance.
#import sagemaker
#from sagemaker import get_execution_role
#role = sagemaker.get_execution_role()
#print(role)

arn:aws:iam::603012210694:role/service-role/AmazonSageMaker-ExecutionRole-20210304T123661


In [14]:
#############################################

In [17]:
#import os
#train_dir = 'train'
#if not os.path.exists(train_dir):
#    os.makedirs(train_dir)

#test_dir = 'test'
#if not os.path.exists(train_dir):
#    os.makedirs(test_dir)

In [12]:
# giving dir location only

train_features_output_path = os.path.join('/opt/ml/processing/train', 'train_features.csv')
##train_features_output_path = os.path.join(train_dir, 'train_features.csv')

train_labels_output_path = os.path.join('/opt/ml/processing/train', 'train_labels.csv')
##train_labels_output_path = os.path.join(test_dir, 'train_labels.csv')

In [13]:
# giving dir location only

test_features_output_path = os.path.join('/opt/ml/processing/test', 'test_features.csv')
##test_features_output_path = os.path.join(train_dir, 'test_features.csv')

test_labels_output_path = os.path.join('/opt/ml/processing/test', 'test_labels.csv')
##test_labels_output_path = os.path.join(test_dir, 'test_labels.csv')

In [21]:
# Now saving files to path define above

y_train.to_csv(train_labels_output_path, header=False, index=False)
y_test.to_csv(test_labels_output_path, header=False, index=False)

Running a processing script

In [22]:
################# Only run bellow codes #######################

In [25]:
import sagemaker
from sagemaker.sklearn.processing import SKLearnProcessor
sklearn_processor = SKLearnProcessor(
                    framework_version='0.20.0',
                    role=sagemaker.get_execution_role(),
                    instance_type='ml.t3.medium',
                    instance_count=1)

In [28]:
from sagemaker.processing import ProcessingInput,ProcessingOutput
sklearn_processor.run(
    code='preprocessing.py',
    inputs=[ProcessingInput(
    source='bank-additional-full.csv',
    # Our data in S3
    destination='/opt/ml/processing/input')],
    outputs=[ProcessingOutput(
    source='/opt/ml/processing/train',
    output_name='train_data'),ProcessingOutput(
    source='/opt/ml/processing/test',
    output_name='test_data')],
    arguments=['--train-test-split-ratio', '0.2'])


Job Name:  sagemaker-scikit-learn-2021-03-06-07-09-07-540
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-603012210694/sagemaker-scikit-learn-2021-03-06-07-09-07-540/input/input-1/bank-additional-full.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-603012210694/sagemaker-scikit-learn-2021-03-06-07-09-07-540/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-603012210694/sagemaker-scikit-learn-2021-03-06-07-09-07-540/output/train_data', 'LocalPath': '/opt/ml/processin

UnexpectedStatusException: Error for Processing job sagemaker-scikit-learn-2021-03-06-07-09-07-540: Failed. Reason: AlgorithmError: See job logs for more information